In [1]:
import pandas as pd
import os

In [76]:
counter = 0
newCsv = pd.DataFrame()
directory = 'datasets-b3'

for file in os.listdir(directory):
    counter = counter+1
    csv = pd.read_csv(directory+'/'+file)
    if(counter == 1):
        newCsv['Data'] = csv['Data']
        
    name = os.path.splitext(file)[0]
    newCsv[name] = csv['Fechamento']

newCsv.to_csv('datasets/data.csv')
print('New file CSV successfully created')

New file CSV successfully created
